# Swarm Ochestration

With AG2, you can initiate a Swarm Chat similar to OpenAI's [Swarm](https://github.com/openai/swarm). In this orchestration, there are two main features:
- **Headoffs**: Agents can transfer control to another agent via function calls, enabling smooth transitions within workflows.
- **Context Variables**: Agents can dynamically update shared variables through function calls, maintaining context and adaptability throughout the process.


Instead of sending a task to a sinlge LLM agent, you can send it to a swarm of agents. Each agent in the swarm can decide whether to handoff the task to another agent. 
When the last active agent's response is only a string (it doesn't suggest tool call or handoff), the chat will be terminated.

We now introduce the main components that need to be used to create a swarm chat.

## Create `SwarmAgent`
All the agents passed to the swarm chat should be instances of this class. `SwarmAgent` is very similar to `AssistantAgent`, but it has some additional features to support function registration and handoffs.

### Initialization
When creating a `SwarmAgent`, you can pass in a list of functions. These functions will be converted to schemas to be passed to the LLMs, and you don't need to worry about registering the functions for execution. You can also pass back a `SwarmResult` class, where you can return a value, the next agent to call and update context variables at the same time.

**Notes for creating the function calls**
- For input arguments, you must define the type of the argument, otherwise the registration will fail (e.g. `arg_name: str`). 
- If your function requires access or modification of the context variables, you must pass in `context_variables: dict` as one argument. This argument will not be visible to the LLM (removed when registering the function schema). But when being called, the global context variables will be passed in by the swarm chat.
- The docstring of the function will be used as the prompt. So make sure to write a clear description.
- The function name will be used as the tool name.

```python
def func_1(arg1: str, arg2: int) -> str:
    """Doc String"""
    return "response"

def transfer_to_some_agent() -> SwarmAgent:
    return SwarmAgent(...)

def func_3(context_variables: dict, arg2: str) -> SwarmResult:
    # ... update context variables
    context_variables["key"] = "value"
    return SwarmResult(value="value", agent=<swarmagent object>, context_variables=context_variables)

SwarmAgent(
    ..., # same arguments as AssistantAgent
    functions=[func_1, transfer_to_some_agent, func_3] # a new parameter that allows passing a list of functions
)
```

### Registering Handoffs
While you can create an function to decide what next agent to call, we provide a quick way to register the handoff use `ON_CONDITION`. We will craft this transition function and add it to the LLM config directly.

```python
agent_2 = SwarmAgent(...)
agent_3 = SwarmAgent(...)

# Register the handoff
agent_1 = SwarmAgent(...)
agent_1.handoff(hand_to=[ON_CONDITION(agent_2, "condition_1"), ON_CONDITION(agent_3, "condition_2")])

# This is equivalent to:
def transfer_to_agent_2():
    """condition_1"""
    return agent_2

def transfer_to_agent_3():
    """condition_1"""
    return agent_3
    
agent_1 = SwarmAgent(..., functions=[transfer_to_agent_2, transfer_to_agent_3])
# You can also use agent_1.add_functions to add more functions after initialization
```

### AFTER_WORK

When the last active agent's response doesn't suggest a tool call or handoff, the chat will terminate by default. However, you can register an `AFTER_WORK` handoff to define a fallback agent if you don't want the chat to end at this agent. At the swarm chat level, you also pass in an `AFTER_WORK` handoff to define the fallback mechanism for the entire chat.
If this parameter is set for both the agent and the chat, we will prioritize the agent's setting.

Besides fallback to an agent, we provide 3 `AfterWorkOption`:
- `TERMINATE`: Terminate the chat
- `STAY`: Stay at the current agent
- `REVERT_TO_USER`: Revert to the user agent. Only if an user agent is passed in when initializing. (See below for more details)

```python
agent_1 = SwarmAgent(...)

# Register the handoff
agent_1.handoff(hand_to=[
    ON_CONDITION(...), 
    ON_CONDITION(...),
    AFTER_WORK(agent_4) # Fallback to agent_4 if no handoff is suggested
])

agent_2.handoff(hand_to=[AFTER_WORK(TERMINATE)]) # Terminate the chat if no handoff is suggested
```

## Initialize SwarmChat with `initiate_swarm_chat`

After a set of swarm agents are created, you can initiate a swarm chat by calling `initiate_swarm_chat`.

```python
chat_history, context_variables, last_active_agent = initiate_swarm_chat(
    init_agent=agent_1, # the first agent to start the chat
    agents=[agent_1, agent_2, agent_3], # a list of agents
    messages=[{"role": "user", "content": "Hello"}], # a list of messages to start the chat
    user_agent=user_agent, # optional, if you want to revert to user agent
    context_variables={"key": "value"} # optional, initial context variables
)
```

### How is context variables updated

### When to pass in a user agent






In [ ]:
from autogen.agentchat.contrib.swarm_agent import SwarmAgent, SwarmResult

llm_config = "Setup your config here"


def update_context_1(context_variables: dict) -> str:
    context_variables["1"] = True
    return SwarmResult(
        value="success", context_variables=context_variables
    )  # Update context variables and return success


def update_context_2_and_transfer_to_3(context_variables: dict) -> str:
    context_variables["2"] = True
    return SwarmResult(
        value="success", context_variables=context_variables, agent=agent_3
    )  # Update context variables, return success, and transfer to agent_3


def update_context_3(context_variables: dict) -> str:
    context_variables["3"] = True
    return SwarmResult(
        value="success", context_variables=context_variables
    )  # Update context variables and return success


agent_1 = SwarmAgent(
    name="Agent_1",
    system_message="You are Agent 1, first, call the function to update context 1, and transfer to Agent 2",
    llm_config=llm_config,
    functions=[update_context_1],
)

agent_2 = SwarmAgent(
    name="Agent_2",
    system_message="You are Agent 2, call the function that updates context 2 and transfer to Agent 3",
    llm_config=llm_config,
    functions=[update_context_2_and_transfer_to_3],
)

agent_3 = SwarmAgent(
    name="Agent_3",
    system_message="You are Agent 3, first, call the function to update context 3, and then reply TERMINATE",
    llm_config=llm_config,
    functions=[update_context_3],
)

agent_1.hand_off(agent_2, condition="handoff to agent 2")  # register handoff using hand_off method.

## Use initiate_swarm_chat

To get response from the swarm agent, you can use the `initiate_swarm_chat` function. This function will return the response from the agent and the updated context variables.

```python
chat_history, context_variables, last_agent = initialize_swarm_chat(
    init_agent, # which agent to start with
    messages, # list of messages
    agents, # pass in all the swarm agents
    max_rounds, # maximum number of rounds
    context_variables, # initial context variables
)
```


In [ ]:
from autogen.agentchat.contrib.swarm_agent import initialize_swarm_chat

context_variables = {"1": False, "2": False, "3": False}
chat_result, context_variables, last_agent = initialize_swarm_chat(
    init_agent=agent_1,
    agents=[agent_1, agent_2, agent_3],
    messages="start",
    context_variables=context_variables,
)
print(context_variables)